In [288]:
# Library Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import math

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [290]:
# Reading the dataset
TextFileReader = pd.read_csv('./data/vehicles.csv', chunksize=10000)
dt = pd.concat(TextFileReader, ignore_index=True)

In [291]:
def findMissing(data, threshold = 5, to_print = True) -> list:
    to_remove = []
    missing_count = 0
    for c in data.columns:
        missing = data.shape[0] - data[c].describe().loc['count']
        if missing>0:
            missing_count += 1
            missing_precent = int(missing)*100/len(data)
            if to_print:
                print('-----------------------')
                print(c)
                print('-----------------------')
                print(data[c].describe())
                print(f'\nMissing Values: {int(missing)}\n')
                print(f'Percentage of Missing Values: {missing_precent}\n')
            if missing_precent <= threshold:
                to_remove.append(c)
    if missing_count == 0:
        print("There's no missing data!!")
    return to_remove

In [292]:
print(dt.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426880 entries, 0 to 426879
Data columns (total 26 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            426880 non-null  int64  
 1   url           426880 non-null  object 
 2   region        426880 non-null  object 
 3   region_url    426880 non-null  object 
 4   price         426880 non-null  int64  
 5   year          425675 non-null  float64
 6   manufacturer  409234 non-null  object 
 7   model         421603 non-null  object 
 8   condition     252776 non-null  object 
 9   cylinders     249202 non-null  object 
 10  fuel          423867 non-null  object 
 11  odometer      422480 non-null  float64
 12  title_status  418638 non-null  object 
 13  transmission  424324 non-null  object 
 14  VIN           265838 non-null  object 
 15  drive         296313 non-null  object 
 16  size          120519 non-null  object 
 17  type          334022 non-null  object 
 18  pain

In [293]:
# Deleting county column since all the values are missing
dt = dt.drop('county',  axis=1)

### Basic data cleaning

In [294]:
cols_for_removal = findMissing(dt, 5, False)
cols_for_removal

['year',
 'manufacturer',
 'model',
 'fuel',
 'odometer',
 'title_status',
 'transmission',
 'image_url',
 'description',
 'lat',
 'long',
 'posting_date']

In [295]:
print(dt.shape)
dt = dt.dropna(subset=cols_for_removal)
print(dt.shape)

(426880, 25)
(383487, 25)


In [298]:
for i in dt.columns:
    print(i)

id
url
region
region_url
price
year
manufacturer
model
condition
cylinders
fuel
odometer
title_status
transmission
VIN
drive
size
type
paint_color
image_url
description
state
lat
long
posting_date
row_items_missing


## Features we can use for search filtering
<ol>
<li> region/state
<li> price
<li> year
<li> manufacturer
<li> model
<li> condition
<li> fuel
<li> odometer
<li> transmission
<li> drive
<li> type
<li> size
</ol>

In [299]:
filter_items = [
    "region",
    "state",
    "price",
    "year",
    "manufacturer",
    "model",
    "condition",
    "fuel",
    "odometer",
    "transmission",
    "drive",
    "type",
    "size",
]

In [300]:
for col in filter_items:
    missing = dt.shape[0] - dt[col].describe().loc['count']
    if missing>0:
        print(col)
        print('-'*10)
        print(dt[col].describe())
        print('-'*5)
        print(f'missing values: {missing}')
        print('-'*30)

condition
----------
count     231391
unique         6
top         good
freq      114699
Name: condition, dtype: object
-----
missing values: 152096
------------------------------
drive
----------
count     269327
unique         3
top          4wd
freq      120840
Name: drive, dtype: object
-----
missing values: 114160
------------------------------
type
----------
count     301093
unique        13
top        sedan
freq       79618
Name: type, dtype: object
-----
missing values: 82394
------------------------------
size
----------
count        107917
unique            4
top       full-size
freq          57401
Name: size, dtype: object
-----
missing values: 275570
------------------------------


# Working with column: condition

In [301]:
print(dt['condition'].value_counts())

condition
good         114699
excellent     89212
like new      19946
fair           5910
new            1093
salvage         531
Name: count, dtype: int64


In [302]:
dtt1 = dt[dt['condition'] == 'good']
dtt2 = dt[dt['condition'] == 'fair']

#### Imputing missing Condition values based on Year and Odometer

In [303]:
features_affecting_condition = ['year', 'odometer']

In [304]:
for i in features_affecting_condition:
    print(i)
    print('-'*5)
    print(dtt1[i].describe())
    print('-'*10)
    print(dtt2[i].describe())
    print('-'*30)

year
-----
count    114699.000000
mean       2012.312801
std           8.847056
min        1913.000000
25%        2009.000000
50%        2015.000000
75%        2018.000000
max        2022.000000
Name: year, dtype: float64
----------
count    5910.000000
mean     1998.792893
std        12.306189
min      1928.000000
25%      1997.000000
50%      2002.000000
75%      2006.000000
max      2022.000000
Name: year, dtype: float64
------------------------------
odometer
-----
count    1.146990e+05
mean     8.049082e+04
std      1.845884e+05
min      0.000000e+00
25%      2.187750e+04
50%      5.029300e+04
75%      1.212545e+05
max      1.000000e+07
Name: odometer, dtype: float64
----------
count    5.910000e+03
mean     2.141005e+05
std      6.227506e+05
min      0.000000e+00
25%      1.220000e+05
50%      1.689390e+05
75%      2.114538e+05
max      1.000000e+07
Name: odometer, dtype: float64
------------------------------


#### Year -> mean(2012, 1999) = 2006
#### Odometer -> mean of means = 1110748
#### if year<2006 and odometer>1.1 10^6 then condition = fair else good


In [305]:
def impute_condition(row):
    if row['condition'] is np.nan:
        if row['year'] < 2006:
            if row['odometer'] > 1110748:
                return 'fair'
            else:
                return 'good'
        else:
            if row['odometer'] > 1110748:
                return 'fair'
            else:
                return 'good'
    else:
        return row['condition']

In [306]:
dt['condition'] = dt.apply(impute_condition, axis=1)

In [307]:
dt['condition'].unique()

array(['good', 'excellent', 'fair', 'like new', 'new', 'salvage'],
      dtype=object)

# Working with column: Drive

In [308]:
for col in ['drive']:
    missing = dt.shape[0] - dt[col].describe().loc['count']
    if missing>0:
        print(col)
        print('-'*10)
        print(dt[col].describe())
        print('-'*5)
        print(f'missing values: {missing}')
        print('-'*30)

drive
----------
count     269327
unique         3
top          4wd
freq      120840
Name: drive, dtype: object
-----
missing values: 114160
------------------------------


In [309]:
dt['drive'].value_counts()

drive
4wd    120840
fwd     97789
rwd     50698
Name: count, dtype: int64

In [310]:
dt[['drive']] = dt[['drive']].fillna(value='fwd')

In [311]:
dt['drive'].value_counts()

drive
fwd    211949
4wd    120840
rwd     50698
Name: count, dtype: int64

# Working with column: VIN, Paint Color

In [312]:
findMissing(dt)

-----------------------
cylinders
-----------------------
count          223023
unique              8
top       6 cylinders
freq            84745
Name: cylinders, dtype: object

Missing Values: 160464

Percentage of Missing Values: 41.843400167411154

-----------------------
VIN
-----------------------
count                241738
unique               107439
top       1FMJU1JT1HEA52352
freq                    261
Name: VIN, dtype: object

Missing Values: 141749

Percentage of Missing Values: 36.963182585068076

-----------------------
size
-----------------------
count        107917
unique            4
top       full-size
freq          57401
Name: size, dtype: object

Missing Values: 275570

Percentage of Missing Values: 71.85901999285504

-----------------------
type
-----------------------
count     301093
unique        13
top        sedan
freq       79618
Name: type, dtype: object

Missing Values: 82394

Percentage of Missing Values: 21.48547408386725

-----------------------
paint_c

[]

In [313]:
dt['VIN'].fillna('missing', inplace=True)

In [314]:
dt['paint_color'].fillna('white', inplace=True)

# Working with column: Size

In [315]:
findMissing(dt)

-----------------------
cylinders
-----------------------
count          223023
unique              8
top       6 cylinders
freq            84745
Name: cylinders, dtype: object

Missing Values: 160464

Percentage of Missing Values: 41.843400167411154

-----------------------
size
-----------------------
count        107917
unique            4
top       full-size
freq          57401
Name: size, dtype: object

Missing Values: 275570

Percentage of Missing Values: 71.85901999285504

-----------------------
type
-----------------------
count     301093
unique        13
top        sedan
freq       79618
Name: type, dtype: object

Missing Values: 82394

Percentage of Missing Values: 21.48547408386725



[]

In [316]:
dt['size'].value_counts()

size
full-size      57401
mid-size       31158
compact        16661
sub-compact     2697
Name: count, dtype: int64

In [317]:
l1 = dt['size'].value_counts().to_dict()
print(l1)

{'full-size': 57401, 'mid-size': 31158, 'compact': 16661, 'sub-compact': 2697}


In [318]:
dt = dt.reset_index(drop=True)
missing_size = dt[dt['size'].isna() == True].index.to_list()

impute_data = {
    'full-size': missing_size[0:146576],
    'mid-size': missing_size[146576:226139],
    'compact': missing_size[226139:268683],
    'sub-compact': missing_size[268683:len(missing_size)]
}

In [319]:
for k,v in impute_data.items():
    for i in v:
        dt['size'].iloc[i] = k

# Working with column: Type

In [320]:
findMissing(dt)

-----------------------
cylinders
-----------------------
count          223023
unique              8
top       6 cylinders
freq            84745
Name: cylinders, dtype: object

Missing Values: 160464

Percentage of Missing Values: 41.843400167411154

-----------------------
type
-----------------------
count     301093
unique        13
top        sedan
freq       79618
Name: type, dtype: object

Missing Values: 82394

Percentage of Missing Values: 21.48547408386725



[]

In [321]:
dt['type'].value_counts()

type
sedan          79618
SUV            69891
pickup         40413
truck          31301
other          18549
coupe          16964
hatchback      14949
wagon          10011
van             7224
convertible     6820
mini-van        4530
offroad          549
bus              274
Name: count, dtype: int64

In [322]:
type_vc = dt['type'].value_counts().to_dict()
avail = dt['type'].describe()['count']

for k,v in type_vc.items():
    type_vc[k] = math.ceil(round((((v/avail)*len(dt)) - v),2))
type_vc

{'sedan': 21788,
 'SUV': 19126,
 'pickup': 11059,
 'truck': 8566,
 'other': 5076,
 'coupe': 4643,
 'hatchback': 4091,
 'wagon': 2740,
 'van': 1977,
 'convertible': 1867,
 'mini-van': 1240,
 'offroad': 151,
 'bus': 75}

In [323]:
missing_type = dt[dt['type'].isna() == True].index.to_list()

impute_type_data = {
    'sedan': missing_type[0:21788],
    'SUV': missing_type[21788:40914],
    'pickup': missing_type[40914:51973],
    'truck': missing_type[51973:60539],
    'other': missing_type[60539:65615],
    'coupe': missing_type[65615:70258],
    'hatchback': missing_type[70258:74349],
    'wagon': missing_type[74349:77089],
    'van': missing_type[77089:79066],
    'convertible': missing_type[79066:80933],
    'mini-van': missing_type[80933:82173],
    'offroad': missing_type[82173:82324],
    'bus': missing_type[82324:],
}

In [324]:
for k,v in impute_type_data.items():
    for i in v:
        dt['type'].iloc[i] = k

# Working with column: Cylinders

In [ ]:
findMissing(dt)

-----------------------
cylinders
-----------------------
count          223023
unique              8
top       6 cylinders
freq            84745
Name: cylinders, dtype: object

Missing Values: 160464

Percentage of Missing Values: 41.843400167411154



[]

In [ ]:
dt['cylinders'].value_counts()

cylinders
6 cylinders     84745
4 cylinders     69547
8 cylinders     64506
5 cylinders      1522
10 cylinders     1291
other             799
3 cylinders       505
12 cylinders      108
Name: count, dtype: int64

In [ ]:
impute_cylinder_data = {}

for k in impute_type_data.keys():
    impute_cylinder_data[k] = dt['cylinders'][dt['type']==k].describe()['top']

for k,v in impute_cylinder_data.items():
    dt['cylinders'][(dt['type'] == k) & dt['cylinders'].isna() == True] = v

In [ ]:
findMissing(dt)

There's no missing data!!


[]

# Imputation Complete

In [ ]:
# dt.to_csv('./Data/clean_vehicles.csv')